<h1 align="center"><font size="5">CONTENT-BASED FILTERING</font></h1>

In [ ]:
import pandas as pd

### Input Dataset


In [ ]:
movies_data = 'https://github.com/clemsonciti/Introduction-to-Hadoop-data/blob/master/ml-latest-small/movies.csv'

### Preprocessing - Menentukan NaN Values

In [ ]:
missing_values = ['na','--','?','-','None','none','non']

### Preprocessing - Dataset dibaca dengan Pandas Dataframe

In [ ]:
movies_df = pd.read_csv(movies_data, na_values=missing_values)

In [ ]:
print('Shape:',movies_df.shape)
movies_df

Movies_df Shape: (9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


### Pisahkan Tahun menjadi kolom terpisah dari Judul dan Split tiap Genre menjadi Array




In [ ]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,(1995)


In [ ]:
# Hilangkan Kurung pada tahun
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995


In [ ]:
# Hilangkan Tahun pada Judul
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995


In [ ]:
## Whitespace harus didelete
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [ ]:
# Genres akan dipisah dan dibentuk kedalam array
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


### Missing Value Check

In [ ]:
movies_df.isna().sum()

movieId     0
title       0
genres      0
year       13
dtype: int64

In [ ]:
movies_df.year.fillna(0, inplace=True)

In [ ]:
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995


One Hot Encode : Memungkinkan tiap kategori memiliki kolomnya sendiri dan berisi nilai 1 dan 0 untuk menunjukan apakah kategori tersebut terpakai atau tidak (Misal terdapat kategori Adventure, Thriller, dan Comedy. Ada user Reyhan yang menyukai film Adventure, maka dalam tabel yang tersedia, hanya Kolom Adventure yang bernilai 1 dan kolom lainnya bernilai 0



In [ ]:
movies_with_genres = movies_df.copy(deep=True)


dump = []
for index, row in movies_df.iterrows():
    dump.append(index)
    for genre in row['genres']:
        movies_with_genres.at[index, genre] = 1

print(len(dump) == len(movies_df))

movies_with_genres.head(3)

True


,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Untuk NaN value, akan digantikan sebagai 0
movies_with_genres = movies_with_genres.fillna(0)
movies_with_genres.head(3)

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Content Based recommender System

Step 1: Buat suatu profile user

In [ ]:
Reyhan_movie_ratings = [
            {'title':'Watchmen', 'rating':3.9},
            {'title':'Menhunter', 'rating':3.9},
            {'title':'Mission Impossible', 'rating':4},
            {'title':"Mummy, The", 'rating':3},
            {'title':'Exorcist, The', 'rating':4.8},
            {'title':'Waiting to Exhale', 'rating':3.9},
            {'title':'Avengers, The', 'rating':4.5},
            {'title':'Jumanji', 'rating':5.0}
         ] 
Reyhan_movie_ratings = pd.DataFrame(Reyhan_movie_ratings)
Reyhan_movie_ratings

,title,rating
0,Watchmen,3.9
1,Menhunter,3.9
2,Mission Impossible,4.0
3,"Mummy, The",3.0
4,"Exorcist, The",4.8
5,Waiting to Exhale,3.9
6,"Avengers, The",4.5
7,Jumanji,5.0


In [ ]:
# Melakukan Merge dengan table movies_df
Reyhan_movie_Id = movies_df[movies_df['title'].isin(Reyhan_movie_ratings['title'])]

Reyhan_movie_ratings = pd.merge(Reyhan_movie_Id, Reyhan_movie_ratings)

Reyhan_movie_ratings

,movieId,title,genres,year,rating
0,2,Jumanji,"[Adventure, Children, Fantasy]",1995,5.0
1,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,3.9
2,1997,"Exorcist, The","[Horror, Mystery]",1973,4.8
3,2153,"Avengers, The","[Action, Adventure]",1998,4.5
4,89745,"Avengers, The","[Action, Adventure, Sci-Fi, IMAX]",2012,4.5
5,2617,"Mummy, The","[Action, Adventure, Comedy, Fantasy, Horror, T...",1999,3.0
6,2633,"Mummy, The","[Horror, Romance]",1932,3.0
7,2634,"Mummy, The",[Horror],1959,3.0
8,60684,Watchmen,"[Action, Drama, Mystery, Sci-Fi, Thriller, IMAX]",2009,3.9


In [ ]:
#Drop kolom yang tidak diperlukan
Reyhan_movie_ratings = Reyhan_movie_ratings.drop(['genres','year'], 1)

Reyhan_movie_ratings

,movieId,title,rating
0,2,Jumanji,5.0
1,4,Waiting to Exhale,3.9
2,1997,"Exorcist, The",4.8
3,2153,"Avengers, The",4.5
4,89745,"Avengers, The",4.5
5,2617,"Mummy, The",3.0
6,2633,"Mummy, The",3.0
7,2634,"Mummy, The",3.0
8,60684,Watchmen,3.9


Step 2: Mempelajari preferensi kesukaan User, Join Tabel film User dengan tabel Movies yang telah di One-Hot-Encoder

In [ ]:
Reyhan_genres_df = movies_with_genres[movies_with_genres.movieId.isin(Reyhan_movie_ratings.movieId)]
Reyhan_genres_df

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1472,1997,"Exorcist, The","[Horror, Mystery]",1973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1611,2153,"Avengers, The","[Action, Adventure]",1998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1972,2617,"Mummy, The","[Action, Adventure, Comedy, Fantasy, Horror, T...",1999,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1983,2633,"Mummy, The","[Horror, Romance]",1932,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,2634,"Mummy, The",[Horror],1959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6797,60684,Watchmen,"[Action, Drama, Mystery, Sci-Fi, Thriller, IMAX]",2009,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7693,89745,"Avengers, The","[Action, Adventure, Sci-Fi, IMAX]",2012,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
Reyhan_genres_df.reset_index(drop=True, inplace=True)

Reyhan_genres_df.drop(['movieId','title','genres','year'], axis=1, inplace=True)

Reyhan_genres_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Step 3: Membangun Preferensi Kesukaan user

In [ ]:
print('Shape of Reyhan_movie_ratings is:',Reyhan_movie_ratings.shape)
print('Shape of Reyhan_genres_df is:',Reyhan_genres_df.shape)

Shape of Reyhan_movie_ratings is: (9, 3)
Shape of Reyhan_genres_df is: (9, 20)


In [ ]:
# Dengan Konsep Matriks Dot, akan diketahui user lebih suka Film dengan Genre apa
Reyhan_profile = Reyhan_genres_df.T.dot(Reyhan_movie_ratings.rating)

Reyhan_profile

Adventure             17.9
Animation              0.0
Children               5.0
Comedy                 8.4
Fantasy                9.5
Romance                6.9
Drama                  6.9
Action                15.9
Crime                  0.0
Thriller               7.5
Horror                15.3
Mystery                7.8
Sci-Fi                 6.9
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   6.9
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Step 4: Deploy Content-Based Recommender System.

In [ ]:
movies_with_genres = movies_with_genres.set_index(movies_with_genres.movieId)

movies_with_genres.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,,,,,
1,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movies_with_genres.drop(['movieId','title','genres','year'], axis=1, inplace=True)

movies_with_genres.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Setelah kita mengetahui Genre film apa saja yang disukai user (Peringkat Genre/Reyhan Profile), maka akan digunakan konsep Matriks Dot antaran Peringkat Genre dengan seluruh film yang ada di dataframe, sehingga bisa didapatkan rekomendasi film yang akan ditonton berdasarkan Profile User

In [ ]:
# Matriks Dot
recommendation_table_df = (movies_with_genres.dot(Reyhan_profile)) / Reyhan_profile.sum()

Urutkan dari Nilai Terbesar



In [ ]:
recommendation_table_df.sort_values(ascending=False, inplace=True)

# Menampilkan 20 Film Teratas
recommendation_table_df.head(20)

movieId
81132     0.693647
2617      0.648390
72165     0.648390
71999     0.622280
36509     0.620540
43932     0.607485
1215      0.583116
52462     0.577894
117646    0.575283
170827    0.575283
164226    0.575283
6902      0.564839
88125     0.564839
2414      0.553525
546       0.553525
4956      0.552654
60471     0.552654
31804     0.547433
85261     0.530896
2735      0.517842
dtype: float64

In [ ]:
copy = movies_df.copy(deep=True)

copy = copy.set_index('movieId', drop=True)

top_20_index = recommendation_table_df.index[:20].tolist()

recommended_movies = copy.loc[top_20_index, :]

recommended_movies

,title,genres,year
movieId,,,
81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
2617,"Mummy, The","[Action, Adventure, Comedy, Fantasy, Horror, T...",1999
72165,Cirque du Freak: The Vampire's Assistant,"[Action, Adventure, Comedy, Fantasy, Horror, T...",2009
71999,Aelita: The Queen of Mars (Aelita),"[Action, Adventure, Drama, Fantasy, Romance, S...",1924
36509,"Cave, The","[Action, Adventure, Horror, Mystery, Sci-Fi, T...",2005
43932,Pulse,"[Action, Drama, Fantasy, Horror, Mystery, Sci-...",2006
1215,Army of Darkness,"[Action, Adventure, Comedy, Fantasy, Horror]",1993
52462,Aqua Teen Hunger Force Colon Movie Film for Th...,"[Action, Adventure, Animation, Comedy, Fantasy...",2007
117646,Dragonheart 2: A New Beginning,"[Action, Adventure, Comedy, Drama, Fantasy, Th...",2000
